In [1]:
from astropy.table import Table
from scipy.stats import zscore
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import sys
import os
from scipy.stats import gaussian_kde
import numpy as np
import json
import urllib
import re

In [2]:
df_h = Table.read('/content/drive/MyDrive/BCU Classification/table-4LAC-DR3-h.fits')
df_h = df_h.to_pandas()
df_h = df_h[df_h['Flags']==0]

df_l = Table.read('/content/drive/MyDrive/BCU Classification/table-4LAC-DR3-l.fits')
df_l = df_l.to_pandas()
df_l = df_l[df_l['Flags']==0]

In [3]:
df_h.head()

,Source_Name,DataRelease,RAJ2000,DEJ2000,GLON,GLAT,Signif_Avg,Flux1000,Unc_Flux1000,Energy_Flux100,...,HE_EPeak,Unc_HE_EPeak,HE_nuFnuPeak,Unc_HE_nuFnuPeak,nu_syn,nuFnu_syn,Variability_Index,Frac_Variability,Unc_Frac_Variability,Highest_energy
0,b'4FGL J0001.2+4741',1,0.3126,47.685902,114.250198,-14.338059,4.692600,1.259796e-10,2.877197e-11,1.499454e-12,...,NaN,NaN,-inf,-inf,1.000000e+14,3.716456e-13,25.313953,0.675882,0.312850,65.575203
1,b'4FGL J0001.2-0747',1,0.3151,-7.797100,89.032722,-67.305008,23.782486,7.471219e-10,5.041939e-11,8.339171e-12,...,796.297852,766.797424,8.686245e-07,7.492923e-08,9.120108e+13,1.936770e-12,46.780693,0.406565,0.111870,86.975197
2,b'4FGL J0001.4-0010',3,0.3717,-0.169900,96.891991,-60.491329,4.900018,1.082246e-10,2.914440e-11,1.231385e-12,...,15475.512695,18471.714844,1.494930e-07,8.294155e-08,3.609939e+12,2.473256e-13,9.272764,0.000000,10.000000,30.056200
3,b'4FGL J0001.5+2113',1,0.3815,21.218300,107.649353,-40.167652,54.653084,1.347354e-09,6.172809e-11,2.555889e-11,...,65.680008,18.915735,8.844835e-06,8.229393e-07,1.584893e+14,1.152471e-12,1910.935791,0.996138,0.213478,-inf
5,b'4FGL J0001.8-2153',3,0.4647,-21.886499,55.184696,-77.572540,4.578068,9.047530e-11,2.442527e-11,1.061114e-12,...,6281.065430,2626.445068,1.545364e-07,5.278574e-08,1.659588e+13,9.716011e-13,24.557972,0.902851,0.380107,-inf


In [4]:
df_l.head()

,Source_Name,DataRelease,RAJ2000,DEJ2000,GLON,GLAT,Signif_Avg,Flux1000,Unc_Flux1000,Energy_Flux100,...,HE_EPeak,Unc_HE_EPeak,HE_nuFnuPeak,Unc_HE_nuFnuPeak,nu_syn,nuFnu_syn,Variability_Index,Frac_Variability,Unc_Frac_Variability,Highest_energy
0,b'4FGL J0012.0+7043',1,3.0032,70.732597,119.661102,8.111637,9.391647,4.022406e-10,4.989643e-11,5.202780e-12,...,NaN,NaN,-inf,-inf,5.754399e+12,5.482871e-13,18.997129,0.381762,0.192456,-inf
1,b'4FGL J0014.7+5801',1,3.6867,58.027199,118.074745,-4.498106,10.829500,1.763550e-10,4.287831e-11,2.571515e-12,...,30467.089844,17695.964844,5.071456e-07,1.101557e-07,0.000000e+00,0.000000e+00,11.195417,0.099530,0.650253,160.817001
2,b'4FGL J0014.8+6118',1,3.7212,61.303699,118.557297,-1.257189,17.677473,7.848316e-10,7.590142e-11,6.474739e-12,...,5067.017578,1501.950317,9.392241e-07,9.292127e-08,2.089296e+13,1.480430e-12,12.274307,0.000000,10.000000,-inf
3,b'4FGL J0015.6+5551',1,3.9071,55.863602,117.888100,-6.657064,13.679805,3.372334e-10,4.716959e-11,4.016745e-12,...,NaN,NaN,-inf,-inf,5.688530e+15,7.878908e-12,11.952563,0.136779,0.318575,167.578003
6,b'4FGL J0035.9+5950',1,8.9823,59.833401,120.982048,-2.979620,68.426636,3.081422e-09,9.592344e-11,3.706086e-11,...,98809.250000,65159.152344,5.909368e-06,5.170865e-07,1.122018e+17,1.493479e-11,252.022751,0.494519,0.109907,372.923004


In [5]:
classification_df = pd.concat([df_h, df_l], ignore_index=True)
classification_df['CLASS'] = classification_df['CLASS'].str.decode('utf-8')
classification_df['CLASS'] = classification_df['CLASS'].str.upper()
classification_df = classification_df[classification_df['CLASS'].isin(['BLL', 'FSRQ', 'BCU'])]

selected_columns = ['PL_Index', 'nu_syn', 'LP_Index', 'Pivot_Energy', 'Frac_Variability', 'Variability_Index', 'nuFnu_syn', 'CLASS']
target = 'CLASS'
feature_columns = [ele for ele in selected_columns if ele is not 'CLASS']


classification_df = classification_df[selected_columns]

classification_df

<>:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-5-6a33a8dc7af4>:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  feature_columns = [ele for ele in selected_columns if ele is not 'CLASS']


,PL_Index,nu_syn,LP_Index,Pivot_Energy,Frac_Variability,Variability_Index,nuFnu_syn,CLASS
0,2.271696,1.000000e+14,2.254081,2420.575195,0.675882,25.313953,3.716456e-13,BCU
1,2.116692,9.120108e+13,2.078927,1721.603882,0.406565,46.780693,1.936770e-12,BLL
2,1.939160,3.609939e+12,1.661223,4307.082520,0.000000,9.272764,2.473256e-13,BLL
3,2.654060,1.584893e+14,2.514159,329.773743,0.996138,1910.935791,1.152471e-12,FSRQ
4,1.876663,1.659588e+13,1.716620,4429.934082,0.902851,24.557972,9.716011e-13,BCU
...,...,...,...,...,...,...,...,...
3169,2.071567,3.311315e+13,2.042366,5897.854004,0.328630,14.602383,1.006315e-12,BCU
3170,2.226648,0.000000e+00,1.962429,3114.226318,1.071415,34.235767,0.000000e+00,BCU
3171,2.045913,0.000000e+00,2.019132,5066.525391,0.337927,22.359163,0.000000e+00,BCU
3172,1.810977,1.566751e+16,1.736923,1910.885742,0.201833,58.688427,1.279994e-11,BLL


In [8]:
bcu_samples = classification_df[classification_df['CLASS'].isin(['BCU'])]
classification_df = classification_df[~classification_df['CLASS'].isin(['BCU'])]

print(classification_df.shape)
print(bcu_samples.shape)

(2005, 8)
(1115, 8)


In [9]:
class_mapping = {'BLL': 0, 'FSRQ': 1}
classification_df['CLASS'] = classification_df['CLASS'].map(class_mapping)

<ipython-input-9-23a26e19f24f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df['CLASS'] = classification_df['CLASS'].map(class_mapping)


In [10]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder

In [11]:
transformer = make_column_transformer(
    (StandardScaler(),
        feature_columns)
)

In [12]:
features = classification_df.drop([target], axis=1)
labels = classification_df[target]

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.50, random_state=42)

In [15]:
X_train.to_csv('train_samples.csv', index=False)
X_val.to_csv('val_samples.csv', index = False)
X_test.to_csv('test_samples.csv', index = False)
y_train.to_csv('train_labels.csv', index = False)
y_test.to_csv('test_labels.csv', index = False)
y_val.to_csv('val_labels.csv', index = False)
bcu_samples.to_csv('bcu_samples.csv', index = False)

In [16]:
transformer.fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)
X_val = transformer.transform(X_val)
bcu_samples = transformer.transform(bcu_samples)

In [17]:
np.save('X_train.npy', X_train)
np.save('X_val.npy', X_val)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_val.npy', y_val)
np.save('y_test.npy', y_test)
np.save('bcu_samples.npy', bcu_samples)

In [18]:
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')
bcu_samples = np.load('bcu_samples.npy')

In [19]:
bll, fsrq = np.bincount(y_train)
print(bll)
print(fsrq)

1053
551


In [20]:
X_train.shape,y_train.shape

((1604, 7), (1604,))

In [21]:
X_train[0]

array([ 0.85295165, -0.09814737,  0.71171683, -0.7151801 , -0.27865052,
       -0.17254569,  0.11640763], dtype=float32)

## Autoencoder Pretext

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np

# Define the self-supervised model architecture
input_layer = Input(shape=(7,))
encoder = Dense(128, activation='relu')(input_layer)
encoder = Dense(64, activation='relu')(encoder)
encoded = Dense(32, activation='relu')(encoder)
decoder = Dense(64, activation='relu')(encoded)
decoder = Dense(128, activation='relu')(decoder)
decoded = Dense(7, activation='linear')(decoder)  # Reconstruction layer

# Create the encoder model (used for self-supervised task)
encoder_model = Model(inputs=input_layer, outputs=encoded)

# Create the full self-supervised model (including reconstruction loss)
self_supervised_model = Model(inputs=input_layer, outputs=decoded)

# Compile the self-supervised model with a loss function (reconstruction loss)
self_supervised_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the self-supervised model on the data
self_supervised_model.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val))

# Extract encoded features from the encoder model
encoded_features_train = encoder_model.predict(X_train)
encoded_features_val = encoder_model.predict(X_val)

# Now you can use the encoded features as inputs for a classification head
classification_input_layer = Input(shape=(32,))  # Use the appropriate size of encoded features
classification_output_layer = Dense(1, activation='sigmoid')(classification_input_layer)

classification_model = Model(inputs=classification_input_layer, outputs=classification_output_layer)
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the classification model on the encoded features
classification_model.fit(encoded_features_train, y_train, epochs=20, batch_size=32, validation_data=(encoded_features_val, y_val))


Epoch 1/50
51/51 [==============================] - 2s 11ms/step - loss: 0.6230 - val_loss: 2.5573
Epoch 2/50
51/51 [==============================] - 0s 4ms/step - loss: 0.1121 - val_loss: 0.3910
Epoch 3/50
51/51 [==============================] - 0s 4ms/step - loss: 0.0245 - val_loss: 0.2909
Epoch 4/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0212 - val_loss: 0.5374
Epoch 5/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0186 - val_loss: 0.1965
Epoch 6/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0227 - val_loss: 0.2296
Epoch 7/50
51/51 [==============================] - 0s 4ms/step - loss: 0.0129 - val_loss: 0.0534
Epoch 8/50
51/51 [==============================] - 0s 5ms/step - loss: 0.0099 - val_loss: 0.0810
Epoch 9/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0099 - val_loss: 0.0879
Epoch 10/50
51/51 [==============================] - 0s 3ms/step - loss: 0.0128 - val_loss: 0.0303
Epoch 11/50
51/51 

## Contrastive Classification


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np


# Create pairs of data samples
def create_pairs(data, labels):
    pairs, pair_labels = [], []
    num_samples = data.shape[0]
    for i in range(num_samples):
        positive_idx = np.where(labels == labels[i])[0]
        negative_idx = np.where(labels != labels[i])[0]

        positive_sample = data[np.random.choice(positive_idx)]
        negative_sample = data[np.random.choice(negative_idx)]

        pairs.append([data[i], positive_sample])
        pair_labels.append(1)  # Positive pair

        pairs.append([data[i], negative_sample])
        pair_labels.append(0)  # Negative pair

    return np.array(pairs), np.array(pair_labels)

pairs_train, pair_labels_train = create_pairs(X_train, y_train)
pairs_val, pair_labels_val = create_pairs(X_val, y_val)

# Define the contrastive classification model architecture
input_a = Input(shape=(7,))
input_b = Input(shape=(7,))
shared_dense = Dense(64, activation='relu')


encoded_a = shared_dense(input_a)
encoded_b = shared_dense(input_b)

merged = Concatenate()([encoded_a, encoded_b])
output_layer = Dense(1, activation='sigmoid')(merged)

contrastive_model = Model(inputs=[input_a, input_b], outputs=output_layer)

# Compile the contrastive classification model
contrastive_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the contrastive classification model on the pairs
contrastive_model.fit([pairs_train[:, 0], pairs_train[:, 1]], pair_labels_train,
                      epochs=50, batch_size=32,
                      validation_data=([pairs_val[:, 0], pairs_val[:, 1]], pair_labels_val))


Epoch 1/50
101/101 [==============================] - 2s 6ms/step - loss: 0.6787 - accuracy: 0.5907 - val_loss: 0.6769 - val_accuracy: 0.6350
Epoch 2/50
101/101 [==============================] - 0s 5ms/step - loss: 0.6665 - accuracy: 0.6153 - val_loss: 0.6600 - val_accuracy: 0.6425
Epoch 3/50
101/101 [==============================] - 0s 4ms/step - loss: 0.6641 - accuracy: 0.6163 - val_loss: 0.6587 - val_accuracy: 0.6375
Epoch 4/50
101/101 [==============================] - 0s 4ms/step - loss: 0.6628 - accuracy: 0.6188 - val_loss: 0.6537 - val_accuracy: 0.6350
Epoch 5/50
101/101 [==============================] - 0s 4ms/step - loss: 0.6624 - accuracy: 0.6191 - val_loss: 0.6495 - val_accuracy: 0.6475
Epoch 6/50
101/101 [==============================] - 0s 4ms/step - loss: 0.6615 - accuracy: 0.6150 - val_loss: 0.6561 - val_accuracy: 0.6225
Epoch 7/50
101/101 [==============================] - 0s 4ms/step - loss: 0.6612 - accuracy: 0.6172 - val_loss: 0.6572 - val_accuracy: 0.6275
Epoch 

In [31]:
# Extract features from the contrastive classification model
feature_extraction_model = Model(inputs=contrastive_model.input[0], outputs=encoded_a)  # Use one branch as feature extractor
features_train = feature_extraction_model.predict(X_train)
features_val = feature_extraction_model.predict(X_val)

# Build a classification model
classification_input_layer = Input(shape=(64,))  # Adjust the input shape to match the size of the extracted features
classification_output_layer = Dense(1, activation='sigmoid')(classification_input_layer)

classification_model = Model(inputs=classification_input_layer, outputs=classification_output_layer)
classification_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the classification model on your labeled data
classification_model.fit(features_train, y_train, epochs=20, batch_size=32, validation_data=(features_val, y_val))

7/7 [==============================] - 0s 2ms/step
Epoch 1/20
51/51 [==============================] - 1s 6ms/step - loss: 0.7129 - accuracy: 0.4420 - val_loss: 0.7499 - val_accuracy: 0.5900
Epoch 2/20
51/51 [==============================] - 0s 3ms/step - loss: 0.5967 - accuracy: 0.8148 - val_loss: 0.5898 - val_accuracy: 0.9100
Epoch 3/20
51/51 [==============================] - 0s 2ms/step - loss: 0.5156 - accuracy: 0.8778 - val_loss: 0.4827 - val_accuracy: 0.9050
Epoch 4/20
51/51 [==============================] - 0s 3ms/step - loss: 0.4609 - accuracy: 0.8897 - val_loss: 0.4285 - val_accuracy: 0.9050
Epoch 5/20
51/51 [==============================] - 0s 3ms/step - loss: 0.4224 - accuracy: 0.8928 - val_loss: 0.3948 - val_accuracy: 0.9000
Epoch 6/20
51/51 [==============================] - 0s 2ms/step - loss: 0.3936 - accuracy: 0.8953 - val_loss: 0.3692 - val_accuracy: 0.9100
Epoch 7/20
51/51 [==============================] - 0s 3ms/step - loss: 0.3713 - accuracy: 0.8971 - val_loss: